In [1]:
#..............................................................................#
#                          LIBRERIAS NECESARIAS                                #
#..............................................................................#
import pandas as pd
import netCDF4 as nc
import numpy as np
import cv2
import os
import errno
import geopandas as gpd
import time
from pyproj import Geod
from shapely.geometry import shape, Polygon, Point, MultiPoint, box

In [2]:
cuenca = input("Nombre de la cuenca: ")
direccion_database = 'D:/'+cuenca+'_complete'

Nombre de la cuenca: NA


In [3]:
def crear_carp(direccion):
    try:
        os.mkdir(direccion)
    except OSError as e:
        if e.errno != errno.EEXIST:
            raise

In [4]:
crear_carp(direccion_database)
lista = pd.read_csv('D:/listado'+cuenca+'.dat', header =  None)


In [5]:
def data_lec(direccion,datos):
    names = ['date', 'time', 'lat', 'lon', 'MWS', 'CPSL','ERMWS', 'R34', 'R50', 'R64', 'R100', 'R']
    return pd.read_csv(direccion+'/'+datos, sep=",", skip_blank_lines=True, header = None, names = names)

In [59]:
for archivo in lista.to_numpy():
    print(archivo[0])

AL012000.dat
AL012001.dat
AL012004.dat
AL012005.dat
AL012006.dat
AL012007.dat
AL012008.dat
AL012009.dat
AL012010.dat
AL012011.dat
AL012012.dat
AL012013.dat
AL012014.dat
AL012015.dat
AL012016.dat
AL012018.dat
AL012020.dat
AL022002.dat
AL022004.dat
AL022005.dat
AL022007.dat
AL022010.dat
AL022011.dat
AL022012.dat
AL022013.dat
AL022015.dat
AL022016.dat
AL022018.dat
AL022019.dat
AL032001.dat
AL032002.dat
AL032003.dat
AL032004.dat
AL032005.dat
AL032010.dat
AL032014.dat
AL032016.dat
AL032017.dat
AL032018.dat
AL032019.dat
AL032020.dat
AL042000.dat
AL042001.dat
AL042003.dat
AL042005.dat
AL042006.dat
AL042007.dat
AL042008.dat
AL042009.dat
AL042011.dat
AL042012.dat
AL042013.dat
AL042014.dat
AL042016.dat
AL052000.dat
AL052001.dat
AL052002.dat
AL052005.dat
AL052007.dat
AL052008.dat
AL052009.dat
AL052010.dat
AL052011.dat
AL052012.dat
AL052014.dat
AL052016.dat
AL052019.dat
AL062002.dat
AL062004.dat
AL062005.dat
AL062006.dat
AL062007.dat
AL062008.dat
AL062013.dat
AL062017.dat
AL062018.dat
AL062019.dat

In [6]:
def formato_fecha(df1):
    df = pd.to_datetime(df1['date'], format='%Y%m%d').apply(lambda x: pd.Series([x.year,x.month,x.day], index = ['yy', 'mm', 'dd']))
    return pd.concat([df,df1], axis = 1)
def formato_hora(df):
    return df.assign(time = (df['time']/100).astype(int))

In [7]:
def cambiar_sufijos(lon1):
    if lon1[-1] == 'W':
        lon1 = lon1.replace("W","")
        lon1 = float(lon1) * -1
    else:
        lon1 = lon1.replace("E","").replace("N","")
        lon1 = float(lon1)
    return lon1

In [8]:
data = data_lec('D:/S7/'+cuenca,'AL012010.dat')

In [9]:
TC = (data
     .pipe(formato_fecha)
     .pipe(formato_hora)
     .assign(lon = data.lon.apply(lambda lon: cambiar_sufijos(lon)))
     .assign(lat = data.lat.apply(lambda lat: cambiar_sufijos(lat)))
     .drop(columns = ['MWS','CPSL','ERMWS','R50','R64','R100'])
     )
TC_concat = TC.copy()

In [10]:
timestamp = (TC_concat[['yy','mm','dd','time']]
             .astype('str')
             .apply(lambda x: x[0]+ x[1].zfill(2) + x[2].zfill(2) + x[3].zfill(2), axis = 1))
center_position = TC_concat[['lon','lat']].apply(lambda x: [x[0],x[1]], axis = 1)

In [11]:
TC_concat = TC_concat.assign(timestamp = timestamp).assign(center_position = center_position)

In [12]:
# LEER IMAGEN 
def abrir_imagen(timestamp):
    ir_image = nc.Dataset('D:/IR/merg_'+ timestamp +'_4km-pixel.nc4.nc4')
    return np.array(ir_image.variables['Tb'][0,:,:] - 273.15)

In [13]:
IR_img = abrir_imagen(TC_concat.timestamp[0])

In [14]:
pic1 = IR_img.copy()

In [15]:
def correccion_valor_minimo(image):
    temp_min = np.min(image[np.where(image != np.min(image))])
    image[np.where(image == np.min(image))] = temp_min - 1
    return image 

In [16]:
IR_img_corregida = correccion_valor_minimo(pic1)

In [17]:
# SEGMENTACION DE IMAGEN 
def binarizar_imagen(img, umbral: float):
    imagen = img.copy()
    pixel_mayor_umbral = np.where(imagen>umbral)
    pixel_valor_nulo = np.where(imagen == np.min(imagen))
    imagen[pixel_mayor_umbral or pixel_valor_nulo] = 0
    pixel_valido = np.where(imagen != 0)
    imagen[pixel_valido] = 255
    return imagen

In [18]:
IR_img_binarizada = binarizar_imagen(IR_img_corregida, -40)
# np.unique(IR_img_binarizada)

In [19]:
def crear_circulo(centro, radio):
    angulo = np.arange(0,np.pi*2,0.01)
    coor_x = radio*np.cos(angulo)+centro[0]
    coor_y = radio*np.sin(angulo)+centro[1]
    return (coor_x,coor_y)

In [21]:
def poligono_circulo(coordenadas): 
    return Polygon(list(zip(coordenadas[0],coordenadas[1])))

In [20]:
contorno_campo_vientos = crear_circulo(TC_concat.center_position[0],TC_concat.R[0]/111.1)

In [22]:
poligono_campo_vientos = poligono_circulo(contorno_campo_vientos)

In [68]:
TC_concat['poly_rout'] = (TC_concat[['center_position','R']]
                          .apply(lambda x: crear_circulo(x[0], x[1]/111.1),axis = 1)
                          .apply(poligono_circulo)
                         )

In [23]:
(contornos,_) = cv2.findContours(np.uint8(IR_img_binarizada), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

In [24]:
np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)
df_contornos = pd.DataFrame(contornos, columns = ['array'])

In [25]:
df_contornos['len'] = df_contornos.array.apply(lambda x: len(x))

In [26]:
df_contornos_sel = df_contornos[df_contornos.len > 10]

In [27]:
def contornos_numpy(array):
    nubM = []
    for c in array: nubM.append(c[0].tolist())
    return np.array(nubM)

In [345]:
# contornos_umbral = df_contornos_sel.array.apply(lambda x: contornos_numpy(x))

In [28]:
selcon = df_contornos_sel.array.apply(lambda x: contornos_numpy(x)).to_frame(name = 'geometry')

In [29]:
def convcoor(nube, dlat= 0.036388397, dlon = 0.036392212):
    lonc1 = np.round(np.transpose(((dlon*nube[:,0])-130)),2)
    latc1 = np.round((dlat*nube[:,1]),2)
    return list(zip(lonc1,latc1))

In [30]:
selcon['geometry'] = selcon['geometry'].apply(convcoor).apply(Polygon)
selcon = gpd.GeoDataFrame(selcon,geometry='geometry',crs='EPSG:4326')

In [31]:
contornos_dentro_campo_vientos = selcon[selcon['geometry'].apply(lambda nube: nube.intersects(poligono_campo_vientos))]

In [75]:
def gdf_conver(gdf):
    """ Función para convertir una serie de datos al crs EPSG:4326, que
    representa una proyección geografica WGS84"""
    return gpd.GeoDataFrame(index=[0], crs='EPSG:4326', geometry=[gdf])
def gdf_explode(gdf):
    """ Separación (explode) de una GeoDataFrame en columnas"""
    return gdf.explode().reset_index(drop=True).rename(columns={0: 'geometry'})

In [72]:
# PROCEDE LA RECTIFICACION DE LOS DATOS 
def rectificacion(selcon,ctpos,r,polyar):
    rec_inf = Polygon([(-130,0),(-130,15),(-10,15),(-10,0)]) #itcz
    a = rec_inf.intersects(polyar)
    pol_cast = Polygon([(-130,0),(-130,15),(ctpos[0]-r,15),(ctpos[0]-r,ctpos[1]-r),
                        (ctpos[0]+r,ctpos[1]-r),(ctpos[0]+r,15),(-10,15),(-10,0)])
    pol_cast = gpd.GeoDataFrame(index=[0], crs='EPSG:4326', geometry=[pol_cast])
    if a == True:
        filtro2 = gpd.overlay(selcon, pol_cast, how='difference')
        return filtro2
    else:
        if ctpos[1] > 15:
            pol_cast = Polygon([(-130,0),(-130,15.1),(ctpos[0]-r,15.1),(ctpos[0]-r,ctpos[1]-r),
                        (ctpos[0]+r,ctpos[1]-r),(ctpos[0]+r,15.1),(-10,15.1),(-10,0)])
            pol_cast = gpd.GeoDataFrame(index=[0], crs='EPSG:4326', geometry=[pol_cast])
            filtro2 = gpd.overlay(selcon, pol_cast, how='difference')
            return filtro2
        else:
            return selcon

In [77]:
def correccion_por_df_vacio(df, data):
    if df.empty:
        df['geometry'] = None
        df['ID']= (np.ones(df.shape[0])*i).astype(int)
        return df
    else:
        result = rectificacion(df,data.center_position[i],data.R[i]/111.1, data.poly_rout[i])
        result['ID']= (np.ones(result.shape[0])*i).astype(int)
        result = gdf_explode(result)
        indx = result['geometry'].apply(lambda nube: nube.intersects(data.poly_rout[i]))
        return result[indx]

In [79]:
i = 0
contornos_corregidos = correccion_por_df_vacio(contornos_dentro_campo_vientos, TC_concat)

In [ ]:
def cal_pol(data):
    data_base = gpd.GeoDataFrame(columns = ['geometry', 'ID'],
                                 crs = 'epsg:4326')
    for i in range(len(data)):
        result = process_imagen(data,i,-40)
        data_base = pd.concat([data_base, result],
                              axis=0).reset_index(drop=True)
    data_base['area'] = data_base['geometry'].apply(areapoly)
    polygons = data_base.loc[data_base['area']>= 800].reset_index(drop= True)
    return polygons

In [ ]:
data_base = gpd.GeoDataFrame(columns = ['geometry', 'ID'], crs = 'epsg:4326')
contornos_corregidos = correccion_por_df_vacio(contornos_dentro_campo_vientos, TC_concat)
